In [6]:
import tensorflow as tf
from tensorflow import keras

In [7]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

x_train_full, x_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_val = scaler.transform(x_val)
x_test = scaler.transform(x_test)

In [8]:
x_train.shape, x_val.shape, x_test.shape

((11610, 8), (3870, 8), (5160, 8))

In [9]:
model = keras.models.Sequential([
    keras.layers.Dense(30, activation="relu", input_shape=x_train.shape[1: ]),
    keras.layers.Dense(1)
])

model.compile(loss="mean_squared_error", optimizer="sgd")

In [10]:
hist = model.fit(x_train, y_train, epochs=20, 
                validation_data=(x_val, y_val))

Epoch 1/20
363/363 [==============================] - 0s 749us/step - loss: 1.3986 - val_loss: 0.4900
Epoch 2/20
363/363 [==============================] - 0s 470us/step - loss: 0.5341 - val_loss: 0.4362
Epoch 3/20
363/363 [==============================] - 0s 463us/step - loss: 0.5262 - val_loss: 0.4168
Epoch 4/20
363/363 [==============================] - 0s 535us/step - loss: 0.4806 - val_loss: 0.4129
Epoch 5/20
363/363 [==============================] - 0s 473us/step - loss: 0.4372 - val_loss: 0.3893
Epoch 6/20
363/363 [==============================] - 0s 463us/step - loss: 0.4202 - val_loss: 0.3884
Epoch 7/20
363/363 [==============================] - 0s 464us/step - loss: 0.4353 - val_loss: 0.3747
Epoch 8/20
363/363 [==============================] - 0s 522us/step - loss: 0.4136 - val_loss: 0.3790
Epoch 9/20
363/363 [==============================] - 0s 470us/step - loss: 0.4167 - val_loss: 0.3635
Epoch 10/20
363/363 [==============================] - 0s 497us/step - loss: 0.418

In [11]:
mse_test = model.evaluate(x_test, y_test)
mse_test

162/162 [==============================] - 0s 311us/step - loss: 0.4736


0.47358909249305725

In [13]:
x_new = x_test[: 3]
y_pred = model.predict(x_new)
y_pred, y_test[: 3]

(array([[0.81721294],
        [2.283176  ],
        [2.177102  ]], dtype=float32),
 array([0.723, 4.1  , 2.441]))

시퀀셜 API를 사용해 회귀용 MLP를 구축, 학습, 평가, 예측하는 방법은 이전에 분류에서 했던 것과 매우 비슷함.  
다른 점은 출력층이 활성화 함수가 없는 하나의 뉴런(하나의 값을 예측하는 경우)이라는 것과 손실 함수로 평균 제곱 오차를 이용한다는 점임.  
이 데이터셋은 잡음이 많기 때문에 과대적합을 막는 용도로 뉴런 수가 적은 은닉층 하나만 사용했다고 함

---
## 함수형 API를 사용해 복잡한 모델 만들기
순차적이지 않은 신경말을 구현하기 위해 함수형 API 사용

In [19]:
input_ = keras.layers.Input(shape=x_train.shape[1: ])
h1 = keras.layers.Dense(30, activation="relu")(input_)
h2 = keras.layers.Dense(30, activation="relu")(h1)
concat = keras.layers.Concatenate()([input_, h2])
output = keras.layers.Dense(1)(concat)

model1 = keras.Model(inputs=[input_], outputs=[output])

함수형 API를 이용해 **와이드 & 딥 신경망** 을 구현한 모습  
>해당 모델은 입력값의 일부 또는 전체가 출력층에 바로 연결되는 특징을 갖고 있음.

In [22]:
input_a = keras.layers.Input(shape=[5], name="wide_input")
input_b = keras.layers.Input(shape=[6], name="deep_input")
h1 = keras.layers.Dense(30, activation="relu")(input_b)
h2 = keras.layers.Dense(30, activation="relu")(h1)
concat = keras.layers.Concatenate()([input_a, h2])
output = keras.layers.Dense(1)(concat)

model2 = keras.Model(inputs=[input_a, input_b], outputs=[output])

입력의 일부는 깊은 경로(일반적인 피드포워드 경로)로 가고 일부는 짧은 경로(출력층과 바로 이어진)로 가게 설정하기 위해선  
위와 같이 멀티 입력을 설정하면 됨.  
이렇게 모델이 복잡해지면 중요한 층에는 이름을 붙이는 것이 좋음.  
그리고 입력이 나눠지면 fit메서드를 호출할 때도 입력마다 하나씩 행렬의 튜플을 전달해야 함 : **(x_train_a, x_train_b)** 

In [21]:
model1.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.001))
hist = model1.fit(x_train, y_train, epochs=20,
                 validation_data=(x_val, y_val))

Epoch 1/20
363/363 [==============================] - 0s 785us/step - loss: 3.6700 - val_loss: 0.7849
Epoch 2/20
363/363 [==============================] - 0s 547us/step - loss: 0.7737 - val_loss: 0.6756
Epoch 3/20
363/363 [==============================] - 0s 539us/step - loss: 0.6667 - val_loss: 0.6095
Epoch 4/20
363/363 [==============================] - 0s 544us/step - loss: 0.6613 - val_loss: 0.5762
Epoch 5/20
363/363 [==============================] - 0s 542us/step - loss: 0.6236 - val_loss: 0.5505
Epoch 6/20
363/363 [==============================] - 0s 561us/step - loss: 0.5881 - val_loss: 0.5379
Epoch 7/20
363/363 [==============================] - 0s 536us/step - loss: 0.5648 - val_loss: 0.5128
Epoch 8/20
363/363 [==============================] - 0s 538us/step - loss: 0.5504 - val_loss: 0.5063
Epoch 9/20
363/363 [==============================] - 0s 529us/step - loss: 0.5419 - val_loss: 0.4878
Epoch 10/20
363/363 [==============================] - 0s 539us/step - loss: 0.524

In [25]:
model2.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=0.001))

x_train_a, x_train_b = x_train[:, :5], x_train[:, 2:]
x_val_a, x_val_b = x_val[:, :5], x_val[:, 2:]
x_test_a, x_test_b = x_test[:, :5], x_test[:, 2:]

hist = model2.fit((x_train_a, x_train_b), y_train, epochs=20,
                 validation_data=((x_val_a, x_val_b), y_val))

Epoch 1/20
363/363 [==============================] - 0s 758us/step - loss: 3.2910 - val_loss: 1.0994
Epoch 2/20
363/363 [==============================] - 0s 536us/step - loss: 0.9669 - val_loss: 0.7665
Epoch 3/20
363/363 [==============================] - 0s 534us/step - loss: 0.7571 - val_loss: 0.6602
Epoch 4/20
363/363 [==============================] - 0s 538us/step - loss: 0.7040 - val_loss: 0.5995
Epoch 5/20
363/363 [==============================] - 0s 549us/step - loss: 0.6230 - val_loss: 0.5628
Epoch 6/20
363/363 [==============================] - 0s 553us/step - loss: 0.5909 - val_loss: 0.5356
Epoch 7/20
363/363 [==============================] - 0s 577us/step - loss: 0.5702 - val_loss: 0.5142
Epoch 8/20
363/363 [==============================] - 0s 589us/step - loss: 0.5426 - val_loss: 0.4975
Epoch 9/20
363/363 [==============================] - 0s 594us/step - loss: 0.5420 - val_loss: 0.4848
Epoch 10/20
363/363 [==============================] - 0s 583us/step - loss: 0.494

여러개의 출력이 필요한 경우도 많음.  
- 그림에 있는 주요 물체를 분류하고 위치를 알아야 할 때 회귀 작업(물체 중심의 좌표, 너비, 높이)과 분류 작업을 함께 할 때.  
- 동일한 데이터에서 독립적인 여러 작업을 수행할 때. 작업마다 다른 신경망을 각각 학습하는 것이 보통 더 나은 결과를 내지만 신경망이 여러 작업에 걸쳐 유용한 특성을 학습하기 때문에 다중 작업 분류가 가능(예를 들어 한 출력은 사람의 얼굴 표정 분류, 다른 출력은 안경을 썼는지 분류를 할 수 있음)  
- 규제 기법으로 사용하는 경우. 예를 들어 신경망 안에 보조 출력을 추가하여 하위 네트워크가 나머지 네트워크에 의존하지 않고 그 자체로 유용한지 학습하는지 확인

In [28]:
input_a = keras.layers.Input(shape=[5], name="wide_input")
input_b = keras.layers.Input(shape=[6], name="deep_input")
h1 = keras.layers.Dense(30, activation="relu")(input_b)
h2 = keras.layers.Dense(30, activation="relu")(h1)
concat = keras.layers.Concatenate()([input_a, h2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(h2)

model3 = keras.Model(inputs=[input_a, input_b], outputs=[output, aux_output])

In [29]:
model3.compile(loss=["mse", "mse"], loss_weights=[0.9, 0.1], optimizer="sgd")

각 출력은 자신만의 손실함수가 필요함. 따라서 모델을 컴파일할 때 손실의 리스트를 전달해야 함.(하나의 손실만 전달하면 케라스는 모든 출력의 손실함수가 동일하다고 가정함)  
기본적으로 케라스는 나열된 손실을 모두 더하여 최종 손실을 구해 학습에 사용함. 
>보조 출력보다 주 출력에 관심이 더 많다면 주 출력에 더 많은 가중치를 두면 됨.

In [32]:
hist = model3.fit((x_train_a, x_train_b), y_train, epochs=20,
          validation_data=((x_val_a, x_val_b), y_val))

Epoch 1/20
363/363 [==============================] - 0s 693us/step - loss: 0.3731 - main_output_loss: 0.3622 - aux_output_loss: 0.4710 - val_loss: 0.3387 - val_main_output_loss: 0.3269 - val_aux_output_loss: 0.4452
Epoch 2/20
363/363 [==============================] - 0s 605us/step - loss: 0.3705 - main_output_loss: 0.3597 - aux_output_loss: 0.4676 - val_loss: 0.3282 - val_main_output_loss: 0.3171 - val_aux_output_loss: 0.4272
Epoch 3/20
363/363 [==============================] - 0s 625us/step - loss: 0.3694 - main_output_loss: 0.3591 - aux_output_loss: 0.4621 - val_loss: 0.3286 - val_main_output_loss: 0.3179 - val_aux_output_loss: 0.4245
Epoch 4/20
363/363 [==============================] - 0s 632us/step - loss: 0.3671 - main_output_loss: 0.3571 - aux_output_loss: 0.4569 - val_loss: 0.3243 - val_main_output_loss: 0.3136 - val_aux_output_loss: 0.4203
Epoch 5/20
363/363 [==============================] - 0s 622us/step - loss: 0.3650 - main_output_loss: 0.3550 - aux_output_loss: 0.4550 

In [34]:
total_loss, main_loss, aux_loss = model3.evaluate([x_test_a, x_test_b], [y_test, y_test])
total_loss, main_loss, aux_loss

162/162 [==============================] - 0s 702us/step - loss: 0.3209 - main_output_loss: 0.3112 - aux_output_loss: 0.4081


(0.3208684027194977, 0.3111749291419983, 0.40811002254486084)

In [35]:
x_new_a = x_test_a[: 3]
x_new_b = x_test_b[: 3]
y_pred_main, y_pred_aux = model3.predict([x_new_a, x_new_b])
y_pred_main, y_pred_aux

(array([[0.9050157],
        [2.3326054],
        [2.6610227]], dtype=float32),
 array([[0.8577765],
        [2.2640398],
        [2.8308058]], dtype=float32))

---
## 서브클래싱 API로 동적 모델 만들기
시퀀셜 API와 함수형 API 모두 선언적임. 사용할 층과 연결 방식을 먼저 정의해야 하고 그 다음 모델에 데이터를 주입하여 학습이나 예측을 시작할 수 있음. 이 방식에는 장점이 많음.  
- 모델을 저장하거나 복사, 공유가 쉬움.  
- 모델의 구조를 출력하거나 분석하기 좋음.  
- 프레임워크가 크기를 짐작하고 타입을 확인하여 에러를 일찍 발견할 수 있음.(모델에 데이터가 주입되기 전에)  
- 전체 모델이 층으로 구성된 정적 그래프이므로 디버깅하기 쉬움.  
  
하지만 어떤 모델은 반복문을 포함하고 다양한 크기를 다루어야 하며 조건문을 가지는 등 여러가지 동적인 구조를 필요로 함.  
이런 경우 **서브클래싱 API** 가 정담임.

In [36]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.h1 = keras.layers.Dense(units, activation=activation)
        self.h2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)
    
    def call(self, inputs):
        input_a, input_b = inputs
        h1 = self.h1(input_b)
        h2 = self.h2(h1)
        concat = keras.layers.concatenate([input_a, h2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(h2)
        return main_outpur, aux_output

간단히 Model클래스를 상속한 다음 생성자 안에서 필요한 층을 만듦.  
그 다음 **call()** 메서드 안에서 수행하려는 연산을 기술함.  
이 예제는 함수형 API와 비슷하지만 Input클래스의 객체를 만들 필요가 없음. 대신 call()메서드의 input 매개변수를 사용함.  
>주된 차이점은 call()메서드 안에서 원하는 어떤 계산도 사용할 수 있음.  
for문, if문, 텐서플로의 저수준 연산도 사용할 수 있음.  
  
하지만 모델 구조가 call()메서드 안에 숨겨져 있기 때문에 케라스가 쉽게 이를 분석할 수 없다고 함.  
**즉 모델을 저장하거나 복사할 수 없음** (????? 그럼 왜씀??)  
summary()메서드를 호출해도 층의 목록만 나열되고 층 간의 연결 정보를 얻을 수 없음.  
> 그래서 높은 유연성이 필요하지 않다면 그냥 시퀀셜이나 함수형 API 사용하는 것이 좋다고 함

In [37]:
model3.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
deep_input (InputLayer)         [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_16 (Dense)                (None, 30)           210         deep_input[0][0]                 
__________________________________________________________________________________________________
wide_input (InputLayer)         [(None, 5)]          0                                            
__________________________________________________________________________________________________
dense_17 (Dense)                (None, 30)           930         dense_16[0][0]                   
____________________________________________________________________________________________